---
title: Prepared for Gabor's Data Analysis
jupyter: python3
---


### Data Analysis for Business, Economics, and Policy
by Gabor Bekes and  Gabor Kezdi
 
Cambridge University Press 2021

**[gabors-data-analysis.com ](https://gabors-data-analysis.com/)**

 License: Free to share, modify and use for educational purposes. 
 Not to be used for commercial purposes.

### Chapter 18
**CH18 Forecasting daily ticket sales for a swimming pool**

using swim data

version 0.9.0 2025-08-14


In [1]:
# ---------------------------------------------------------
# Frieren's Grimoire: Bringing tools from the void
# ---------------------------------------------------------
import os  # Operating System: The map to our file locations
import sys  # System: To manipulate the path of our journey
import warnings  # Warnings: To silence the minor spirits
from datetime import datetime  # Time: To understand the flow of ages

import numpy as np  # NumPy: The fundamental math of the elves
import pandas as pd  # Pandas: The table-manipulation magic
import seaborn as sns  # Seaborn: To weave beautiful visual illusions
from matplotlib import pyplot as plt  # Matplotlib: The canvas for our art
import pandas_market_calendars as mcal  # Calendars: Knowledge of the holidays
import pyfixest as pf  # PyFixest: High-performance econometric spells
from sklearn.metrics import root_mean_squared_error  # RMSE: The judge of our accuracy

# Silencing warnings to keep the mind clear
warnings.filterwarnings("ignore")

In [2]:
# ---------------------------------------------------------
# Setting the Path: Where does our journey begin?
# ---------------------------------------------------------
# Current script folder - finding our feet
current_path = os.getcwd()
dirname = current_path.split("da_case_studies")[0]

# Defining the locations of our artifacts (Directories)
data_in = dirname + "da_data_repo/swim-transactions/clean/"
data_out = dirname + "da_case_studies/ch18-swim-transactions/"
output = dirname + "da_case_studies/ch18-swim-transactions/output/"
func = dirname + "da_case_studies/ch00-tech-prep/"

# Adding the function directory to our spellbook path
sys.path.append(func)

In [3]:
import py_helper_functions as da

# Set custom color scheme for plots - The Robes of the Mage
sns.set_theme(rc=da.da_theme, palette=da.color)

### 🧝‍♀️ Frieren's Seminar: The Pandas Lexicon - Dates
**Mage Sight (Eye Movement):** Focus on `parse_dates=["date"]`.

In the ancient tongue of CSV, dates are but mere strings (Text). A common apprentice mistake is to leave them as such.
*   **The Artifact:** `pd.read_csv(..., parse_dates=...)`
*   **The Transformation:** `String Class` $\to$ `Timestamp Class`
*   **Why it matters:** Accessing parts of a date (Year/Month) from a String requires slow regex parsing. A `Timestamp` object stores time as a 64-bit integer (nanoseconds since 1970), allowing for instant O(1) retrieval.

**Arch-Mage Note:** If you forget this, your future spells (like `.dt.year`) will fail with an `AttributeError`.


In [4]:
# Reading the scroll (CSV) and interpreting the "date" runes immediately
daily_agg = pd.read_csv(os.path.join(data_in, "swim_work.csv"), parse_dates=["date"])
# daily_agg = pd.read_csv("https://osf.io/download/jcxmk/", parse_dates=["date"])

In [5]:
# The Mage's First Glance: Inspecting the head of the beast
daily_agg.head()

,date,QUANTITY
0,2010-01-01,0
1,2010-01-02,49
2,2010-01-03,31
3,2010-01-04,14
4,2010-01-05,18


### 🧝‍♀️ Frieren's Seminar: The `.dt` Accessor
**Mage Sight:** Look at `daily_agg["date"].dt.year`.

This `.dt` is a specialized "Namespace Accessor".
*   **The Concept:** A Series (column) in Pandas usually holds generic Objects. But if it holds Timestamps, the `.dt` accessor exposes the C-level attributes of the underlying array.
*   **Efficiency:** `daily_agg["date"].year` would fail. You *Must* go through the gate of `.dt`.
*   **Vectorization:** This extraction happens in the C-layer, processing millions of rows instantly, unlike a Python loop.

### 🧝‍♀️ Frieren's Seminar: List Membership
**Mage Sight:** Look at `.isin([6, 7])`.
*   **The Logic:** `x in [6, 7]` is Python. `.isin([6, 7])` is Pandas.
*   **The Difference:** The Pandas version checks the entire column at once, returning a Boolean Array (True/False) of the same length `(N)`.

**Mana Flow:** `Timestamp Column` $\to$ `.dt` $\to$ `Integer Attributes`

In [ ]:
# Extracting time components using the .dt accessor
daily_agg["year"] = daily_agg["date"].dt.year # The Year (e.g., 2015)
daily_agg["quarter"] = daily_agg["date"].dt.quarter # The Quarter (1-4)
daily_agg["month"] = daily_agg["date"].dt.month # The Month (1-12)
daily_agg["day"] = daily_agg["date"].dt.day # The Day of the month (1-31)
daily_agg["dow"] = daily_agg["date"].dt.dayofweek + 1 # Day of Week (Mon=1, Sun=7)

# Identifying Weekends: Is the day Saturday (6) or Sunday (7)?
daily_agg["weekend"] = daily_agg["dow"].isin([6, 7])

### 🧝‍♀️ Frieren's Seminar: Boolean Masks (Bitwise Logic)
**Mage Sight:** Focus on the `&` and `|` symbols.

You might ask: *"why not use `and` / `or`?"*
*   **Python Logic:** `and` checks the "Truthiness" of an entire object.
*   **Bitwise Logic (`&`, `|`):** These operators compare array elements *position-by-position*.
    *   `[True, False] & [True, True] -> [True, False]`
*   **Parentheses:** They are **mandatory** here. `daily_agg["day"] > 15 & ...` would crash because `&` has higher precedence than `>`. The parentheses force the comparison to happen first.

**The Spell:** We are constructing a "Mask" (Screen).
*   **Input:** Multiple Boolean Arrays.
*   **Output:** A single Boolean Array `(N,)`, where True means "School was Closed".

In [ ]:
# Creating the "School Off" mask using Bitwise Logic (&, |)
# Parentheses are critical here to enforce order of operations!
daily_agg["school_off"] = (
    ((daily_agg["day"] > 15) & (daily_agg["month"] == 5) & (daily_agg["day"] <= 30)) # Late May
    | ((daily_agg["month"] == 6) | (daily_agg["month"] == 7)) # June or July (Support Summer)
    | ((daily_agg["day"] < 15) & (daily_agg["month"] == 8)) # Early August
    | ((daily_agg["day"] > 20) & (daily_agg["month"] == 12)) # Late December (Winter Break)
)

In [ ]:
# Creating a simple linear trend for time
daily_agg["trend"] = daily_agg.index + 1

In [ ]:
# Get holiday calendar ----------------------------------

### 🧝‍♀️ Frieren's Seminar: External Knowledge
**Mage Sight:** `mcal.get_calendar("NYSE")`.

Sometimes the data you need involves complexity defined elsewhere (like holiday rules). Be lazy. Use a library that encodes this knowledge (`pandas_market_calendars`).

**The Join Logic:**
*   `.isin(holidays)`: This is effectively a "Left Join" check.
*   **True**: The date exists in the holiday list.
*   **False**: It does not.

In [ ]:
```{python}
nyse = mcal.get_calendar("NYSE")

holidays = nyse.holidays().holidays

daily_agg["isHoliday"] = daily_agg["date"].isin(holidays)

In [ ]:
# A quick glance at the numerical summaries (Distribution Check)
daily_agg.describe()

### 🧝‍♀️ Frieren's Seminar: Broadcast vs. Aggregate
**Mage Sight:** `daily_agg.groupby("month")["QUANTITY"].transform("mean")`

This is a subtle but powerful spell.
1.  **Split:** Break the data into 12 piles (one for each month).
2.  **Apply:** Calculate the `mean` of Quantity for each pile.
    *   *Result so far:* A series of length 12 (Jan=150, Feb=140...).
3.  **Broadcast (`transform`):** This is the key. Instead of returning the 12 summaries, Pandas *pastes* the Jan-Average next to *every* January row in the original dataset.

**Shape Mechanics:**
*   `.agg("mean")` $\to$ Returns shape `(12,)` (Group Summary).
*   `.transform("mean")` $\to$ Returns shape `(N,)` (Original Size).

**Why?** We want to regress "Daily Stats" against "Monthly Averages". To do that, they must be in the same DataFrame.

### Define vars for analysis

### Define vars for analysis

In [ ]:
# 1. Monthly Seasonality (Mean Quantity per Month)
daily_agg["q_month"] = daily_agg.groupby("month")["QUANTITY"].transform("mean")

# 2. Log Transformation (Stabilizing the Variance)
daily_agg["q_ln"] = np.log(daily_agg["QUANTITY"])

# Handling the Void (Inf/-Inf): Replace infinite values with NaN so they don't break the model
daily_agg["q_ln"] = daily_agg["q_ln"].replace([np.inf, -np.inf], np.nan)

# 3. Weekly Seasonality within Months (Mean Quantity per Month-DayOfWeek combination)
daily_agg["tickets"] = daily_agg.groupby(["month", "dow"])["QUANTITY"].transform("mean")

# 4. Log Version of Weekly Seasonality
daily_agg["tickets_ln"] = daily_agg.groupby(["month", "dow"])["q_ln"].transform("mean")

# 5. Abbreviations for visualization (e.g., 'Mon', 'Jan')
daily_agg["dow_abb"] = daily_agg["date"].dt.day_name().str[:3]
daily_agg["month_abb"] = daily_agg["date"].dt.month_name().str[:3]

## Descriptive graphs

### 🧝‍♀️ Frieren's Seminar: The Art of Time-Series Visualization
**Mage Sight:** `sns.lineplot(...)`

A simple spell, but one that requires precise calibration.
*   **The Filter:** We slice time `data[year == 2015]`. Plotting *everything* creates a messy "hairball". Zooming in reveals the rhythm.
*   **The Ticks:** `plt.xticks`. By default, Matplotlib might label the axis with incomprehensible numbers. We force it to speak our language (`%d%b%Y` -> 01Jan2015).

In [ ]:
# Focusing on a single year (2015) to see the daily rhythm
filtered_data = daily_agg[daily_agg["year"] == 2015].reset_index()

# Drawing the line of time
sns.lineplot(data=filtered_data, x="date", y="QUANTITY", linewidth=0.6)

# Calibrating the X-Axis labels (The tick marks)
date_ticks = ["2015-01-01", "2015-04-01", "2015-07-01", "2015-10-01", "2016-01-01"]
plt.xticks(date_ticks, [pd.to_datetime(date).strftime("%d%b%Y") for date in date_ticks])

# Labelling the axes for the observer
plt.xlabel("Date (day)")
plt.ylabel("Daily ticket sales")
plt.show()

In [ ]:
filtered_data = daily_agg.loc[
    (daily_agg.year >= 2010) & (daily_agg.year <= 2014), :
].reset_index()

sns.lineplot(data=filtered_data, x="date", y="QUANTITY", linewidth=0.3)
date_ticks = [
    "2010-01-01",
    "2011-01-01",
    "2012-01-01",
    "2013-01-01",
    "2014-01-01",
    "2015-01-01",
]
plt.xticks(date_ticks, [pd.to_datetime(date).strftime("%d%b%Y") for date in date_ticks])
plt.xlabel("Date (day)")
plt.ylabel("Daily ticket sales")
plt.show()

### 🧝‍♀️ Frieren's Seminar: Reading the Box Plot (The Quartile Grimoire)
**Mage Sight:** `sns.boxplot(...)`.

A Box Plot is a spell that summarizes the *Distribution* of mana (data).
*   **The Box:** The middle 50% of the data (Interquartile Range - IQR). From the 25th percentile to the 75th.
*   **The Line:** The Median (50th percentile). The heart of the distribution.
*   **The Whiskers:** The range of "Normal" data (usually 1.5x IQR).
*   **The Diamonds (Fliers):** Rare events (Outliers).

In [ ]:
# Defining the Order: We force the months to align with the calendar, not the alphabet
month_order = [
    "Jan", "Feb", "Mar", "Apr", "May", "Jun", 
    "Jul", "Aug", "Sep", "Oct", "Nov", "Dec",
]

# Summoning the Box Plot
sns.boxplot(
    data=daily_agg,
    x="month_abb", # Grouping by Month
    y="QUANTITY", # analyzing Ticket Sales
    fliersize=3, # Size of the outlier diamonds
    order=month_order, # Enforcing our custom order
    flierprops={
        "markerfacecolor": da.color[3], # coloring the outliers
        "markeredgecolor": da.color[3],
        "alpha": 0.6,
    },
    boxprops={"facecolor": "white", "edgecolor": da.color[0]},
    whiskerprops={"color": da.color[0]},
    medianprops={"color": da.color[0], "linewidth": 2},
    showcaps=False,
)
plt.yticks(np.arange(0, 400, 100)) # Calibrating the Y-axis
plt.xlabel("Date (month)")
plt.ylabel("Daily ticket sales")
plt.grid(axis="x", color="gray", linestyle="-", linewidth=0.5) # Adding a grid for precision
plt.show()

In [ ]:
# Defining the Order of Days (Monday first!)
dow_order = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]

# Summoning the Box Plot for Days of the Week
sns.boxplot(
    data=daily_agg,
    x="dow_abb",
    y="QUANTITY",
    fliersize=3,
    order=dow_order,
    flierprops={
        "markerfacecolor": da.color[3],
        "markeredgecolor": da.color[3],
        "alpha": 0.6,
    },
    boxprops={"facecolor": "white", "edgecolor": da.color[0]},
    whiskerprops={"color": da.color[0]},
    medianprops={"color": da.color[0], "linewidth": 2},
    showcaps=False,
)
plt.yticks(np.arange(0, 400, 100))
plt.xlabel("Day of the week")
plt.ylabel("Daily ticket sales")
plt.grid(axis="x", color="gray", linestyle="-", linewidth=0.5)
plt.show()

### 🧝‍♀️ Frieren's Seminar: Categorical Data & Ordered Factors
**Mage Sight:** `pd.Categorical(..., ordered=True)`.

Strings like "Mon", "Tue"... have no inherent order to a computer. Alphabetically, "Fri" comes before "Mon". This is chaos.
*   **The Fix:** We define a `Categorical` type with a specific `categories` list.
*   **The Effect:** When we later sort or plot, "Mon" will correctly appear before "Tue", and "Jan" before "Feb", honoring the laws of time rather than the alphabet.

### 🧝‍♀️ Frieren's Seminar: Thermal Vision (The Heatmap)
**Mage Sight:** `pivot(...)` + `heatmap(...)`.

This spell reveals the density of mana across two dimensions (Time X Time).
1.  **Pivot:** We must reshape the data from Long ("Row per day") to Wide ("Rows=Month, Cols=Day"). This is the Matrix format.
2.  **Heatmap:** We apply a color gradients (`viridis_r`) to this matrix. Lighter colors = High Traffic.

In [ ]:
# to check for interactions, look at the heatmap
plt.figure(figsize=(9, 6))

# Enforcing Order on the Dimensions
daily_agg["dow_abb_ordered"] = pd.Categorical(
    daily_agg["dow_abb"], categories=dow_order, ordered=True
)
daily_agg["month_abb_ordered"] = pd.Categorical(
    daily_agg["month_abb"], categories=reversed(month_order), ordered=True
)

# Aggregating: Calculating Mean Tickets for each (Month, Day) pair
agg_data = daily_agg.groupby(["month_abb_ordered", "dow_abb_ordered"], as_index=False)[
    "tickets"
].mean()

# Pivoting: The Transformation to Matrix Form
heatmap_data = agg_data.pivot(
    index="month_abb_ordered", columns="dow_abb_ordered", values="tickets"
)

# Summoning the Heatmap
sns.heatmap(
    heatmap_data,
    cmap="viridis_r", # The color palette (Reversed Viridis)
    linewidths=0.5, # Grid lines
    linecolor="white",
    cbar_kws={"shrink": 0.4, "aspect": 5}, # Configuring the Legend Bar
)

plt.xlabel("Day of the week")
plt.ylabel("Month")
plt.yticks(rotation=0) # Keeping Y-labels horizontal for readability

# Adjusting the Color Bar (The Scale of Magic)
cbar = plt.gca().collections[0].colorbar
cbar.ax.tick_params(labelsize=10)
cbar.set_label("Tickets", fontsize=10)
cbar.set_ticks([50, 100, 150])

plt.show()

In [ ]:
# not in book
# Repeating the Spell for Log-Transformed Data
plt.figure(figsize=(9, 6))
agg_data = daily_agg.groupby(["month_abb_ordered", "dow_abb_ordered"], as_index=False)[
    "tickets_ln"
].mean()

heatmap_data = agg_data.pivot(
    index="month_abb_ordered", columns="dow_abb_ordered", values="tickets_ln"
)

sns.heatmap(
    heatmap_data,
    cmap="viridis_r",
    linewidths=0.5,
    linecolor="white",
    cbar_kws={"shrink": 0.4, "aspect": 5, "label": "Log tickets"},
)

plt.xlabel("Day of the week")
plt.ylabel("Month")
plt.yticks(rotation=0)

cbar = plt.gca().collections[0].colorbar
cbar.ax.tick_params(labelsize=10)

plt.show()

## Prediction

### Creat train/holdout data

### 🧝‍♀️ Frieren's Seminar: The Chronological Split
**Mage Sight:** `daily_agg.loc[...]` with strict year barriers.

In normal magic (standard Machine Learning), we use `train_test_split(shuffle=True)`.
*   **The Forbidden Technique:** Shuffling is forbidden in Time Series.
*   **Why?** If you shuffle, you might train on tomorrow to predict yesterday. This is "Look-ahead Bias".
*   **The Syntax:** We manually slice `.loc[data["year"] < 2016]` (The Past) and `== 2016` (The Future).

**Mana Flow:** `(All Data)` $\to$ `(Past, Future)`
**Arch-Mage Note:** Strict temporal separation prevents data leakage.

In [ ]:
# 1. Preparing the Factor columns (Categories)
factor_cols = ["month", "dow", "isHoliday", "school_off"]
daily_agg[factor_cols] = daily_agg[factor_cols].astype("category")

# 2. The Holdout Set (The Future: 2016)
data_holdout = daily_agg.loc[daily_agg["year"] == 2016, :]

# 3. The Training Set (The Past: 2010-2015)
data_train = daily_agg.loc[daily_agg["year"] < 2016, :]

### 🧝‍♀️ Frieren's Seminar: The Manual Cross-Validation Loop
**Mage Sight:** Focus on the `for` loop and the `.loc` inside it.

Standard libraries (`sklearn.model_selection.KFold`) are often insufficient for complex time structures. We must weave our own Validation Spell.

**The Loop Anatomy:**
1.  **Select the Test Year:** `data[group_col] == test_set` (e.g., Year 2010).
2.  **Select the Training Years:** `data[group_col] != test_set` (All other years).
3.  **Train & Predict:** Fit on the "Other Years", Predict on the "Single Year".
4.  **Accumulate:** `rmse_folds.append(rmse)`.

**Syntax Note (Scope):** `rmse_folds = []` is defined *outside* the loop. The loop mutates it by appending results. This is the classic "Accumulator Pattern".

In [ ]:
# Frieren: A custom spell to iterate through time periods
def inserted_test_set_cv(
    formula: str,
    data: pd.DataFrame,
    group_col: str,
    correct_log_transformation_bias: bool = False,
) -> float:
    """
    Perform inserted test set cross-validation.
    """
    # 1. Input Parsing: Extracting the target variable name (Left side of ~)
    y_variable = formula.split("~")[0].strip()
    
    # 2. Cleaning: Dropping missing values in the target to avoid errors
    data = data.dropna(subset=[y_variable])

    # 3. Identifying the Epochs (Years) to iterate over
    test_sets = data[group_col].unique()
    rmse_folds = [] # The Accumulator for our results

    # 4. The Loop of Time
    for test_set in test_sets:
        # Define Training Data: Everything EXCEPT the current test year
        data_train = data.loc[data[group_col] != test_set, :]
        
        # Define Test Data: ONLY the current test year
        data_test = data.loc[data[group_col] == test_set, :]

        # 5. Training the Model (Fitting the spell)
        model = pf.feols(formula, data=data_train)

        # 6. Forecasting
        y_hat = model.predict(data_test)
        y_true = data_test[y_variable]
        
        # 7. Evaluating Accuracy (RMSE)
        rmse = root_mean_squared_error(y_true, y_hat)

        # 8. Correction for Log-Models (If we predicted log(y), we must convert back)
        if correct_log_transformation_bias:
            # Applying the correction factor exp(sigma^2 / 2)
            y_hat = np.exp(y_hat) * np.exp(rmse**2 / 2)
            y_true = np.exp(y_true)
            # Re-calculating RMSE in the original scale
            rmse = root_mean_squared_error(y_true, y_hat)

        # Storing the result
        rmse_folds.append(rmse)

    # Frieren: aggregating the shards of time into a single truth (Mean Probability)
    return np.mean(rmse_folds)

In [ ]:
# ---------------------------------------------------------
# Frieren's Grimoire: Defining the complexity tiers
# ---------------------------------------------------------

# Model 1: The Simplest Spell (Linear Trend + Month)
model1 = "QUANTITY ~ trend + month"

# Model 2: Adding Weekly Rhythms (+ Day of Week)
model2 = "QUANTITY ~ trend + month + dow"

# Model 3: Injecting External Knowledge (+ Holidays)
model3 = "QUANTITY ~ trend + month + dow + isHoliday"

# Model 4: Handling Special Events (+ School Off Interaction)
# Note: 'school_off*dow' adds both the main effect and the interaction
model4 = "QUANTITY ~ trend + month + dow + isHoliday + school_off*dow"

# Model 5: The Grand Arch-Mage Spell (Full Complexity)
# Adding interactions between Weekend and Month (Summer weekends vs Winter weekends)
model5 = "QUANTITY ~ trend + month + dow + isHoliday + school_off*dow + weekend*month"

In [ ]:
# The Evaluation Ritual: Testing each spell against the flow of time
cross_validated_rmses = []

for model in [model1, model2, model3, model4, model5]:
    # Running the Temporal Cross-Validation
    rmse = inserted_test_set_cv(model, data_train, "year")
    cross_validated_rmses.append(rmse)

In [ ]:
# Model 6: The Logarithmic Alteration
# Sometimes mana flows exponentially. We take the log() to make it linear.
model6 = "q_ln ~ trend + month + dow + isHoliday + school_off*dow"

# We must fix the bias when converting back from Log-World to Real-World
rmse6 = inserted_test_set_cv(
    model6, data_train, "year", correct_log_transformation_bias=True
)
cross_validated_rmses.append(rmse6)

### Use prophet prediction

In [ ]:
from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

### 🧝‍♀️ Frieren's Deciphering: Summoning the Prophet
**The Spell Anatomy:**
* `Prophet(...)`: Initializing the additive model structure (Trend + Seasonality).
* `add_country_holidays`: Injecting the Guild's calendar (US Holidays).

**Mana Flow:** `(ds, y)` $\to$ `(Forecast Model)`
**Arch-Mage Note:** Prophet requires a specific data shape: columns named `ds` (date) and `y` (target). It is strict about this naming convention.

In [ ]:
# Frieren: Initializing the Oracle. Note 'additive' seasonality.
model_prophet = Prophet(
    seasonality_mode="additive", # Adding effects (Trend + Season + Holiday)
    yearly_seasonality="auto", # Detecting annual cycles automatically
    weekly_seasonality="auto", # Detecting weekly cycles
    growth="linear", # Assuming linear growth over time
    daily_seasonality=True, # Accounting for daily patterns
)

# Injecting the knowledge of US Holidays into the Oracle
model_prophet = Prophet.add_country_holidays(model_prophet, "US")

In [ ]:
# Fitting the Oracle to the Past
model_prophet = Prophet.fit(
    model_prophet,
    # Frieren: The Oracle demands 'ds' and 'y' columns. We must comply.
    df=data_train[["date", "QUANTITY"]].rename({"date": "ds", "QUANTITY": "y"}, axis=1),
)

In [ ]:
# Generating the Cross-Validation folds (Temporal Backtesting)
cv_pred = cross_validation(
    model_prophet, 
    initial="365 days", # Training on at least 1 year
    period="365 days", # Moving forward by 1 year at a time
    horizon="365 days" # Predicting 1 year into the future
)

In [ ]:
# Extracting the RMSE from the Oracle's visions
rmse_prophet_cv = performance_metrics(cv_pred, rolling_window=1)["rmse"].values[0]
cross_validated_rmses.append(rmse_prophet_cv)

In [ ]:
# Note: M6 log model rmse is slightly different from book
pd.DataFrame(
    cross_validated_rmses,
    ["M" + str(i) for i in range(1, 6)] + ["M6 (log)", "M7 (Prophet)"],
    columns=["RMSE"],
).round(5)

## Evaluate best model on holdout set

### 🧝‍♀️ Frieren's Deciphering: The Final Verdict
**The Spell Anatomy:**
* `best_model.predict`: We use the model trained on *all* history (2010-2015) to predict the future (2016).
* `root_mean_squared_error`: The judge.

**Mana Flow:** `(Holdout Data)` $\to$ `(Predictions)` $\to$ `(Error Metric)`

In [ ]:
best_model = pf.feols(model5, data=data_train)

data_holdout["y_hat_5"] = best_model.predict(data_holdout)

In [ ]:
rmse_holdout_best = root_mean_squared_error(data_holdout.QUANTITY, data_holdout.y_hat_5)
rmse_holdout_best

### Plot best predictions

In [ ]:
# graph relative RMSE (on holdout) per month

group = data_holdout.sort_values(by=["date"]).groupby("month")
rmse_monthly = pd.DataFrame(
    {
        "date": group["date"].first(),
        "RMSE": group.apply(
            lambda x: root_mean_squared_error(x["QUANTITY"], x["y_hat_5"])
        ),
        "RMSE_norm": group.apply(
            lambda x: root_mean_squared_error(x["QUANTITY"], x["y_hat_5"])
            / np.mean(x["QUANTITY"])
        ),
    }
).reset_index(drop=True)

### Figure 18.7 b)

In [ ]:
ax = sns.barplot(x="date", y="RMSE_norm", data=rmse_monthly)
ax.set_xticklabels(rmse_monthly["date"].dt.strftime("%b"))
ax.set_yticks(np.arange(0, 1.6, 0.5))
ax.set_xlabel("Date (month)")
ax.set_ylabel("RMSE (normalized by monthly sales)")
plt.grid(axis="x", color="gray", linestyle="-", linewidth=0.5)
plt.show()

### 🧝‍♀️ Frieren's Seminar: The Unpivot (Melt)
**Mage Sight:** `dataset.melt(...)`

To plot multiple lines (Actual vs Predicted), plotting libraries (Seaborn/Ggplot) demand **"Tidy" (Long) Data**.
*   **The Problem:** Our data is "Wide". We have separate columns `QUANTITY` and `y_hat`.
*   **The Solution (`melt`):** We collapse these two columns into one.
    *   `variable` column: Contains the names "QUANTITY" or "y_hat".
    *   `value` column: Contains the numbers.

**Method Chaining (`.`):**
Notice the parentheses usage `( ... )`. This allows us to chain multiple operations (`filter` -> `melt` -> `merge` -> `rename`) without creating intermediate variables like `df_temp1`, `df_temp2`. This is cleaner syntax.

In [ ]:
# Frieren: Reshaping the timeline into a long format for the Sea-born Plotter (Seaborn)
plotdata = (
    data_holdout
    .filter(["date", "month", "QUANTITY", "y_hat_5"]) # Select only essential mana streams
    .melt(id_vars=["date", "month"]) # Unpivot: Collapsing Qty and Prediction into one column
    # Re-attaching the original values for reference (Self-Join)
    .merge(data_holdout.filter(["date", "QUANTITY"]), on="date") 
    .merge(data_holdout.filter(["date", "y_hat_5"]), on="date")
    # Renaming for clarity in the legend
    .rename(columns={"QUANTITY": "ymin", "y_hat_5": "ymax"})
    .assign(
        variable=lambda x: x["variable"].map(
            {"QUANTITY": "Actual", "y_hat_5": "Predicted"} # Renaming the labels
        )
    )
)

### Figure 18.6

In [ ]:
sns.lineplot(x="date", y="value", hue="variable", style="variable", data=plotdata)

xticks = pd.to_datetime(
    ["2016-01-01", "2016-03-01", "2016-05-01", "2016-07-01", "2016-09-01", "2016-11-01"]
)
xlabels = ["01Jan2016", "01Mar2016", "01May2016", "01Jul2016", "01Sep2016", "01Nov2016"]
plt.xticks(xticks, labels=xlabels, fontsize=9)
plt.xlim((datetime(2016, 1, 1), datetime(2017, 1, 1)))
plt.xlabel("Date (day)")
plt.ylabel("Daily ticket sales")
plt.legend(bbox_to_anchor=(0.54, 0.77), frameon=False, ncol=2)
plt.show()

### Figure 18.7 a)

In [ ]:
# Frieren: Focusing the mage sight on a single month (August) to inspect the fine details.
plotdata_august = plotdata[plotdata["month"] == 8].reset_index()
sns.lineplot(
    x="date",
    y="value",
    hue="variable",
    style="variable",
    data=plotdata_august,
    linewidth=1.5,
)

plt.fill_between(
    plotdata_august.loc[lambda x: x["variable"] == "Actual", "date"],
    plotdata_august.loc[lambda x: x["variable"] == "Actual", "ymin"],
    plotdata_august.loc[lambda x: x["variable"] == "Actual", "ymax"],
    color=da.color[3],
    alpha=0.2,
)
plt.xticks(
    pd.to_datetime(
        ["2016-08-01", "2016-08-08", "2016-08-15", "2016-08-22", "2016-08-29"]
    ),
    labels=["01Aug", "08Aug", "15Aug", "22Aug", "29Aug"],
)
plt.ylim(0, 150)
plt.xlim((datetime(2016, 8, 1), datetime(2016, 8, 31)))
plt.yticks(np.arange(0, 151, 50))
plt.xlabel("Date (day)")
plt.ylabel("Daily ticket sales")
plt.legend(
    title="", loc="upper left", bbox_to_anchor=(0.54, 0.77), frameon=False, ncol=2
)

plt.show()